In [1]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import pystan
import fbprophet 
import plotly 



#built-in 
import datetime

print('pandas ==',pd.__version__)
print('numpy ==',np.__version__)
print('matplotlib ==',matplotlib.__version__)
print('seaborn ==',sns.__version__)
print('plotly ==', plotly.__version__)
print('pystan ==',pystan.__version__)
print('fbprophet ==',fbprophet.__version__)

pandas == 1.2.2
numpy == 1.19.2
matplotlib == 3.3.4
seaborn == 0.11.1
plotly == 4.11.0
pystan == 2.19.1.1
fbprophet == 0.7.1


# Funções

In [2]:
# =======================================================================================
## FUNÇÕES PARA A IMPORTAÇÃO DOS DADOS E TRANSFORMAR DATAS

def converte_variaveis_em_datas(dataframe):
    for variavel in dataframe.columns:
        if 'DATA' in variavel:
            try:
                dataframe[variavel] = pd.to_datetime(dataframe[variavel], format='%d/%m/%Y')
            
            except:
                print(f'Variável "{variavel}" contém um erro e conversão/formatação')
                pass
        else:
            pass 
    return dataframe


def tira_variaveis_IBGE(dataframe):
    dataframe = dataframe.drop(['IBGE_RES_PR','IBGE_ATEND_PR'], axis='columns')
    return dataframe


def cria_variavel_caso_confirmado_no_dia(dataframe):
    dataframe['CASO_CONFIRMADO_NO_DIA'] = 1
    return dataframe


def replace_nas_variaveis_obito_status(dataframe):

    dataframe['OBITO'] = dataframe['OBITO'].replace('SIM','Sim')
    dataframe['OBITO'] = dataframe['OBITO'].replace('Não','Nao')

    dataframe['OBITO'] = dataframe['OBITO'].replace('Sim',1)
    dataframe['OBITO'] = dataframe['OBITO'].replace('Nao',0)

    dataframe['STATUS'] = dataframe['STATUS'].replace('Recuperado','recuperado')
    dataframe['STATUS'] = dataframe['STATUS'].replace('recuperado', 1)
    dataframe['STATUS'] = dataframe['STATUS'].replace('nan', 0)

    return dataframe
    

def pre_processamento(dataframe):
    dataframe = converte_variaveis_em_datas(dataframe)
    dataframe = tira_variaveis_IBGE(dataframe)
    dataframe = cria_variavel_caso_confirmado_no_dia(dataframe)
    dataframe = replace_nas_variaveis_obito_status(dataframe)
    dataframe = dataframe.set_index('DATA_CONFIRMACAO_DIVULGACAO')
    dataframe = dataframe.sort_index()
    return dataframe

def baixa_base_de_dados_casos_gerais(ano,mes,dia):
    """
    Esta função baixa a base de dados disponível no ano, mes e dia.
    retorna um dataframe do pandas com os dados disponpiveis.
    """
    data = datetime.date(ano, mes, dia)
    ano = str(data.year)

    if mes != 10 and mes != 11 and mes != 12:
        mes = '0'+str(data.month)
    else:
        mes = str(data.month)

    if dia in [1,2,3,4,6,7,8,9]:
        dia = '0'+str(data.day)

    ano_mes = ano+'-'+mes

    if ano == '2020':
        arquivo = f'INFORME_EPIDEMIOLOGICO_{dia}_{mes}_GERAL.csv'
    elif ano == '2021':
        arquivo = f'informe_epidemiologico_{dia}_{mes}_{ano}_geral.csv'
    # Podem acontecer as seguintes variações no site do estado:
    # informe_epidemiologico_{dia}_{mes}_{ano}_geral.csv
    # informe_epidemiologico_{dia}_{mes}_geral.csv
    # INFORME_EPIDEMIOLOGICO_{dia}_{mes}_GERAL.csv



    dominio = 'https://www.saude.pr.gov.br'
    caminho = f'/sites/default/arquivos_restritos/files/documento/{ano_mes}/'
    try:

        url = dominio+caminho+arquivo
        base_de_dados = pd.read_csv(url, sep=';')
        base_de_dados = pre_processamento(base_de_dados)
        
    
    except:
        raise Exception('Não Tem dados disponíveis neste dia.')

    return base_de_dados

# =======================================================================================
## FUNÇÕES PARA SELEÇÃO 

def seleciona_media_movel(dataframe):

    if 'MUN_ATENDIMENTO' in dataframe.columns and 'DATA_CONFIRMACAO_DIVULGACAO' in dataframe.columns:
        dataframe = dataframe.groupby(['MUN_ATENDIMENTO','DATA_CONFIRMACAO_DIVULGACAO'])[['CASO_CONFIRMADO_NO_DIA','OBITO']].sum().rolling(15).mean()[15:]
        return dataframe
    else:
        raise Exception('')




# Importando os Dados 

In [3]:
hoje = datetime.date.today()
informe_covid = baixa_base_de_dados_casos_gerais(hoje.year, hoje.month, hoje.day - 1)

ValueError: day is out of range for month

In [4]:
informe_covid

,SEXO,IDADE_ORIGINAL,MUN_RESIDENCIA,MUN_ATENDIMENTO,LABORATORIO,DATA_DIAGNOSTICO,DATA_INICIO_SINTOMAS,OBITO,DATA_OBITO,DATA_OBITO_DIVULGACAO,STATUS,DATA_RECUPERADO_DIVULGACAO,FONTE_DADO_RECUPERADO,CASO_CONFIRMADO_NO_DIA
DATA_CONFIRMACAO_DIVULGACAO,,,,,,,,,,,,,,
2020-03-12,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-18,15/03/2020,0.0,NaT,NaT,1.0,NaT,NaN,1
2020-03-12,M,15,CURITIBA,CURITIBA,LACEN,2020-03-12,08/03/2020,0.0,NaT,NaT,1.0,NaT,NaN,1
2020-03-12,F,47,CIANORTE,CIANORTE,LACEN,2020-03-12,NaN,0.0,NaT,NaT,1.0,NaT,Notificações,1
2020-03-12,M,43,CURITIBA,CURITIBA,LACEN,2020-03-12,07/03/2020,0.0,NaT,NaT,1.0,NaT,NaN,1
2020-03-12,M,28,SAO PAULO/SP,CURITIBA,Genoprimer,2020-03-16,NaN,0.0,NaT,NaT,1.0,2020-08-05,SMS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-10,F,13,CLEVELANDIA,NaN,"COVID-19, Biologia Molecular",2021-03-09,NaN,NaN,NaT,NaT,NaN,NaT,NaN,1
2021-03-10,F,17,CAMPO MOURAO,CAMPO MOURAO,"COVID-19, Biologia Molecular",2021-03-08,28/02/2021,NaN,NaT,NaT,NaN,NaT,NaN,1
2021-03-10,F,21,LUIZIANA,LUIZIANA,Teste Rápido Antígeno (Imunocromatografia),2021-03-09,07/03/2021,NaN,NaT,NaT,NaN,NaT,NaN,1


In [5]:
def seleciona_cidade(cidade, dataframe):
    """
    Esta função retona todos os registros
    cujo o atendimento ocorreu na cidade 
    passada como parâmetro.
    Retorna um dataframe.
    """
    dados_cidade = dataframe.query(f'MUN_ATENDIMENTO == "{cidade.upper()}"')
    return dados_cidade.reset_index()



In [6]:
move_average_15_por_cidade = informe_covid.groupby(['MUN_ATENDIMENTO','DATA_CONFIRMACAO_DIVULGACAO'])[['CASO_CONFIRMADO_NO_DIA','OBITO']].sum().rolling(14).mean()[14:].round()
casos_por_cidade = informe_covid.groupby(['MUN_ATENDIMENTO','DATA_CONFIRMACAO_DIVULGACAO'])[['CASO_CONFIRMADO_NO_DIA','OBITO']].sum().round()

In [7]:
move_average_15_por_cidade

CASO_CONFIRMADO_NO_DIA  OBITO
MUN_ATENDIMENTO DATA_CONFIRMACAO_DIVULGACAO                               
ABATIA          2020-07-01                                      2.0    0.0
                2020-07-02                                      2.0    0.0
                2020-07-03                                      2.0    0.0
                2020-07-07                                      3.0    0.0
                2020-07-08                                      3.0    0.0
...                                                             ...    ...
XAMBRE          2021-02-26                                      2.0    0.0
                2021-02-27                                      2.0    0.0
                2021-03-02                                      2.0    0.0
                2021-03-04                                      2.0    0.0
                2021-03-07                                      2.0    0.0

[51277 rows x 2 columns]

In [8]:
casos_por_cidade

CASO_CONFIRMADO_NO_DIA  OBITO
MUN_ATENDIMENTO DATA_CONFIRMACAO_DIVULGACAO                               
ABATIA          2020-05-08                                        1    0.0
                2020-05-31                                        1    0.0
                2020-06-05                                        4    0.0
                2020-06-08                                        1    0.0
                2020-06-11                                        1    0.0
...                                                             ...    ...
XAMBRE          2021-02-26                                        3    0.0
                2021-02-27                                        1    0.0
                2021-03-02                                        1    0.0
                2021-03-04                                        2    0.0
                2021-03-07                                        4    0.0

[51291 rows x 2 columns]

In [9]:
def atualiza_dados_diarios():
    hoje = datetime.date.today()
    informe_covid = baixa_base_de_dados_casos_gerais(hoje.year, hoje.month, hoje.day - 1)

    move_average_15_por_cidade = informe_covid.groupby(['MUN_ATENDIMENTO','DATA_CONFIRMACAO_DIVULGACAO'])[['CASO_CONFIRMADO_NO_DIA','OBITO']].sum().rolling(14).mean()[14:].round()
    move_average_15_por_cidade.to_csv('../dados_recentes/cidades_mm15.csv', sep=';')

    casos_por_cidade = informe_covid.groupby(['MUN_ATENDIMENTO','DATA_CONFIRMACAO_DIVULGACAO'])[['CASO_CONFIRMADO_NO_DIA','OBITO']].sum().round()
    casos_por_cidade.to_csv('../dados_recentes/casos_por_cidade.csv', sep=';')    



# Dados de Internações

In [10]:
uri_internacoes = 'https://raw.githubusercontent.com/ConradBitt/covid_parana/main/dados_recentes/INTERNACOES_TRATAMENTO%20DE%20INFEC%C3%87%C3%83O%20PELO_CORONAVIRUS_COVID%2019.csv'

In [11]:
internacoes = pd.read_csv(uri_internacoes,sep=';',
 skiprows = 4,
 skipfooter=14,
 thousands=',',
 engine='python', encoding='iso-8859-1')

In [12]:
internacoes

,Município,2020/Mar,2020/Abr,2020/Mai,2020/Jun,2020/Jul,2020/Ago,2020/Set,2020/Out,2020/Nov,2020/Dez,2021/Jan,Total
0,410045 Altamira do Paraná,-,-,-,-,-,2,-,1,-,-,-,3
1,410060 Alto Paraná,-,-,-,-,-,-,-,-,-,-,1,1
2,410100 Ampére,-,-,-,-,2,3,6,6,6,6,13,42
3,410140 Apucarana,-,12,27,29,69,69,60,38,40,41,3,388
4,410150 Arapongas,-,7,26,43,118,85,60,14,19,110,84,566
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,412800 Ubiratã,-,-,-,1,-,-,5,6,8,26,17,63
124,412810 Umuarama,-,2,7,29,89,82,56,30,55,137,49,536
125,412820 União da Vitória,-,-,3,7,13,11,7,3,20,26,6,96
126,412855 Vera Cruz do Oeste,-,-,-,4,3,-,-,-,1,3,-,11


In [13]:
def pre_processamento_dados_internacoes(dataframe):
    from unidecode import unidecode
    municipios = []
    for municipio in dataframe['Município']:
        municipios.append(unidecode(municipio[7:]).upper())

    dataframe['Município'] = municipios
    dataframe.drop('Total',axis='columns')
    dataframe = dataframe.T.iloc[:-1]
    index = [
        'MUN_ATENDIMENTO', '2020-03', '2020-04',
        '2020-05', '2020-06', '2020-07',
        '2020-08', '2020-09', '2020-10', 
        '2020-11', '2020-12', '2021-01'
        ]

    dataframe.index = index
    dataframe = dataframe.T
    dataframe = dataframe.set_index('MUN_ATENDIMENTO')
    dataframe = dataframe.replace('-',0).astype(int)
    dataframe.columns = pd.to_datetime(dataframe.columns)
    return dataframe.T



In [14]:
def pre_processamento_dados_internacoes(dataframe):
    from unidecode import unidecode
    municipios = []
    for municipio in dataframe['Município']:
        municipios.append(unidecode(municipio[7:]).upper())

    dataframe['Município'] = municipios
    dataframe.drop('Total',axis='columns')
    dataframe = dataframe.T.iloc[:-1]
    index = [
        'MUN_ATENDIMENTO', '2020-03', '2020-04',
        '2020-05', '2020-06', '2020-07',
        '2020-08', '2020-09', '2020-10', 
        '2020-11', '2020-12', '2021-01'
        ]

    dataframe.index = index
    dataframe = dataframe.T
    dataframe = dataframe.set_index('MUN_ATENDIMENTO')
    dataframe = dataframe.replace('-',0).astype(int)
    dataframe.columns = pd.to_datetime(dataframe.columns)
    return dataframe.T

def carrega_internacoes():
    uri_internacoes = 'https://raw.githubusercontent.com/ConradBitt/covid_parana/main/dados_recentes/INTERNACOES_TRATAMENTO%20DE%20INFEC%C3%87%C3%83O%20PELO_CORONAVIRUS_COVID%2019.csv'
    internacoes = pd.read_csv(uri_internacoes,sep=';',skiprows = 4,skipfooter=14,thousands=',', engine='python', encoding='iso-8859-1')
    internacoes = pre_processamento_dados_internacoes(internacoes)

    return internacoes

In [15]:
internacoes = pre_processamento_dados_internacoes(internacoes)


In [19]:
internacoes[['PONTA GROSSA']]

MUN_ATENDIMENTO,PONTA GROSSA
2020-03-01,0
2020-04-01,22
2020-05-01,21
2020-06-01,63
2020-07-01,126
2020-08-01,140
2020-09-01,145
2020-10-01,151
2020-11-01,167
2020-12-01,234


In [161]:
def exibe_internacoes_cidade(dataframe, cidade):
    cidade = cidade.upper()
    internacoes = dataframe[cidade]
    internacoes.rename_axis('Data', inplace=True)

    fig = px.bar(dataframe[cidade], x=internacoes.index, y = f'{cidade}',
     color=f'{cidade}', height=400, labels={f'{cidade}': 'Internados'})
    fig.layout.title.text = f'Internações por COVID-19 - {cidade}'
    fig.layout.xaxis.title.text = 'Mês de internação'
    fig.layout.yaxis.title.text = ''
    fig.update_coloraxes(
        colorbar=dict(title='Internações'),
        colorbar_title_font_size=22,
        colorbar_title_side='top')

    return fig

In [162]:
exibe_internacoes_cidade(internacoes, 'Ponta Grossa').show()

In [122]:
internacoes['PONTA GROSSA']

2020-03-01      0
2020-04-01     22
2020-05-01     21
2020-06-01     63
2020-07-01    126
2020-08-01    140
2020-09-01    145
2020-10-01    151
2020-11-01    167
2020-12-01    234
2021-01-01    204
Name: PONTA GROSSA, dtype: int64

In [5]:
import datetime
import pandas as pd

class InformeCovid():
    """
    Esta classe tem como objetivo carregar os informes
    da covid do estado do paraná. Ao utilizar a função
    carrega_informe() o resultado será um dataframe 
    com o informe do dia, mes e ano.

    Use o método carrega_informe(): para carregar todo o
        informe do paraná.
    
    OU

    Use o método carrega_informe_mm(): para carregar as 
    médias móveis da cidade do paraná. É possível modificar
    o parâmetro move_average para mudar a janela.
    """
    def __init__(self):
        pass

    def carrega_informe(self, data):
        ano = data.year
        mes = data.month
        dia = data.day - 3

        self._data_informe = datetime.date(ano, mes, dia)
        self.dia, self.mes, self.ano = self.__ajusta_data(self._data_informe)
        self.__uri = self.__ajusta_uri(self.dia, self.mes, self.ano)

        try:
            dados_informe_covid = pd.read_csv(self.__uri, sep=';')
            dados_informe_covid = self.__pre_processamento(dados_informe_covid)
            return dados_informe_covid
        except:
            raise Exception(f'Não Tem dados disponíveis neste dia.\n{self.__uri}')

    def carrega_informe_mm(self, data, janela = 14):
        dados = self.carrega_informe(data)
        dados = self.__pre_processamento(dados)
        dados_agrupados = dados.groupby(['MUN_ATENDIMENTO','DATA_CONFIRMACAO_DIVULGACAO'])

        self.informe_mm = dados_agrupados[
            ['CASO_CONFIRMADO',
            'OBITO']
            ].sum().rolling(janela).mean()[janela:]
        
        return self.informe_mm



    def __ajusta_uri(self, dia, mes, ano):
        dia = str(dia)
        mes = str(mes)
        ano = str(ano)

        if dia in ['1','2','3','4','6','7','8','9']:
            dia = '0' + dia

        if ano == '2020':
            arquivo = f'INFORME_EPIDEMIOLOGICO_{dia}_{mes}_GERAL.csv'
        elif ano == '2021':
            arquivo = f'informe_epidemiologico_{dia}_{mes}_{ano}_geral.csv' 

        ano_mes = ano + '-' + mes
        dominio = 'https://www.saude.pr.gov.br'
        caminho = f'/sites/default/arquivos_restritos/files/documento/{ano_mes}/'   
        
        uri = dominio + caminho + arquivo 

        return uri

    
    def __ajusta_data(self, data_informe):
        ano = str(data_informe.year)
        mes = str(data_informe.month)
        dia = str(data_informe.day)
        
        if mes != 10 and mes != 11 and mes != 12:
            mes = '0'+str(data_informe.month)
        else:
            mes = str(data_informe.month)

        if dia in [1,2,3,4,6,7,8,9]:
            dia = '0'+str(data_informe.day)
        
        return dia, mes, ano


    def __pre_processamento(self, dataframe):
        """
        Esta função faz o pré processamento dos dados
        - converte_variaveis_em_datas
        - tira_variaveis_IBGE
        - cria_variavel_caso_confirmado_no_dia(dataframe)
        - replace_nas_variaveis_obito_status(dataframe)
        - usa 'DATA_CONFIRMACAO_DIVULGACAO' como indice
        - ordena pelo índice

        retorna o dataframe.
        """

        dataframe = self.__converte_variaveis_em_datas(dataframe)
        dataframe = self.__tira_variaveis_IBGE(dataframe)
        dataframe =  self.__cria_variavel_caso_confirmado(dataframe)
        dataframe = self.__replace_nas_variaveis_obito_status(dataframe)
        dataframe = dataframe.set_index('DATA_CONFIRMACAO_DIVULGACAO')
        dataframe = dataframe.sort_index()

        return dataframe
        

    def __converte_variaveis_em_datas(self, dataframe):
        for variavel in dataframe.columns:
            if 'DATA' in variavel:
                try:
                    dataframe[variavel] = pd.to_datetime(dataframe[variavel], format='%d/%m/%Y')
                
                except:
                    print(f'Variável "{variavel}" contém um erro e conversão/formatação')
                    pass
            else:
                pass 
        return dataframe


    def __tira_variaveis_IBGE(self, dataframe):
        dataframe = dataframe.drop(['IBGE_RES_PR','IBGE_ATEND_PR'], axis='columns')
        return dataframe


    def __cria_variavel_caso_confirmado(self, dataframe):
        dataframe['CASO_CONFIRMADO'] = 1
        return dataframe


    def __replace_nas_variaveis_obito_status(self, dataframe):

        if 'OBITO' in dataframe.columns:
            dataframe['OBITO'] = dataframe['OBITO'].replace('SIM','Sim')
            dataframe['OBITO'] = dataframe['OBITO'].replace('Não','Nao')
        
            dataframe['OBITO'] = dataframe['OBITO'].replace('Sim',1)
            dataframe['OBITO'] = dataframe['OBITO'].replace('Nao',0)
        else:
            pass
        if 'STATUS' in dataframe.columns:
            dataframe['STATUS'] = dataframe['STATUS'].replace('Recuperado','recuperado')
            dataframe['STATUS'] = dataframe['STATUS'].replace('recuperado', 1)
            dataframe['STATUS'] = dataframe['STATUS'].replace('nan', 0)
        else:
            pass
        return dataframe


    def atualiza_dados_diarios(hoje):
        informe_covid = self.carrega_informe(hoje.year, hoje.month, hoje.day - 1)

        move_average_15_por_cidade = informe_covid.groupby(['MUN_ATENDIMENTO','DATA_CONFIRMACAO_DIVULGACAO'])[['CASO_CONFIRMADO_NO_DIA','OBITO']].sum().rolling(14).mean()[14:].round()
        move_average_15_por_cidade.to_csv('../dados_recentes/cidades_mm15.csv', sep=';')

        casos_por_cidade = informe_covid.groupby(['MUN_ATENDIMENTO','DATA_CONFIRMACAO_DIVULGACAO'])[['CASO_CONFIRMADO_NO_DIA','OBITO']].sum().round()
        casos_por_cidade.to_csv('../dados_recentes/casos_por_cidade.csv', sep=';')    






        

In [6]:
informe = InformeCovid()

In [11]:
hoje = datetime.date.today()
dados = informe.carrega_informe(hoje)

Variável "DATA_INICIO_SINTOMAS" contém um erro e conversão/formatação


In [12]:
dados.head()

,SEXO,IDADE_ORIGINAL,MUN_RESIDENCIA,MUN_ATENDIMENTO,LABORATORIO,DATA_DIAGNOSTICO,DATA_INICIO_SINTOMAS,OBITO,DATA_OBITO,DATA_OBITO_DIVULGACAO,STATUS,DATA_RECUPERADO_DIVULGACAO,FONTE_DADO_RECUPERADO,CASO_CONFIRMADO
DATA_CONFIRMACAO_DIVULGACAO,,,,,,,,,,,,,,
2020-03-12,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-18,15/03/2020,0.0,NaT,NaT,1.0,NaT,NaN,1
2020-03-12,M,15,CURITIBA,CURITIBA,LACEN,2020-03-12,08/03/2020,0.0,NaT,NaT,1.0,NaT,NaN,1
2020-03-12,F,47,CIANORTE,CIANORTE,LACEN,2020-03-12,NaN,0.0,NaT,NaT,1.0,NaT,Notificações,1
2020-03-12,M,43,CURITIBA,CURITIBA,LACEN,2020-03-12,07/03/2020,0.0,NaT,NaT,1.0,NaT,NaN,1
2020-03-12,M,28,SAO PAULO/SP,CURITIBA,Genoprimer,2020-03-16,NaN,0.0,NaT,NaT,1.0,2020-08-05,SMS,1


In [13]:
PG = dados.query('MUN_ATENDIMENTO == "PONTA GROSSA"')

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [20]:
PG.groupby(['DATA_CONFIRMACAO_DIVULGACAO'], as_index=True).sum().reset_index()

,DATA_CONFIRMACAO_DIVULGACAO,IDADE_ORIGINAL,OBITO,STATUS,CASO_CONFIRMADO
0,2020-03-21,33,0.0,1.0,1
1,2020-03-25,91,0.0,2.0,2
2,2020-04-04,125,0.0,2.0,2
3,2020-04-14,61,0.0,1.0,1
4,2020-04-18,62,0.0,1.0,1
...,...,...,...,...,...
310,2021-03-06,14567,4.0,5.0,344
311,2021-03-07,4922,1.0,12.0,121
312,2021-03-08,1542,0.0,1.0,37
313,2021-03-09,7108,8.0,6.0,157


# Número de Habitantes

In [135]:
populacao_pr = """
Posição 	Município 	População
1 	Curitiba 	1 948 626
2 	Londrina 	575 377
3 	Maringá 	430 157
4 	Ponta Grossa 	355 336
5 	Cascavel 	332 333
6 	São José dos Pinhais 	329 058
7 	Foz do Iguaçu 	258 248
8 	Colombo 	246 540
9 	Guarapuava 	182 644
10 	Paranaguá 	156 174
11 	Araucária 	146 214
12 	Toledo 	142 645
13 	Apucarana 	136 234
14 	Campo Largo 	133 865
15 	Pinhais 	133 490
16 	Arapongas 	124 810
17 	Almirante Tamandaré 	120 041
18 	Piraquara 	114 970
19 	Umuarama 	112 500
20 	Cambé 	107 341
21 	Fazenda Rio Grande 	102 004
22 	Sarandi 	97 803
23 	Campo Mourão 	95 488
24 	Francisco Beltrão 	92 216
25 	Paranavaí 	88 922
26  Pato Branco 	83 843
27 	Cianorte 	83 816
28 	Telêmaco Borba 	79 792
29 	Castro 	71 809
30 	Rolândia 	67 383
31 	Irati 	61 088
32 	União da Vitória 	57 913
33 	Ibiporã 	55 131
34 	Marechal Cândido Rondon 	53 495
35 	Prudentópolis 	52 513
36 	Palmas 	51 755
37 	Lapa 	48 410
38 	Cornélio Procópio 	47 842
39 	São Mateus do Sul 	46 705
40 	Medianeira 	46 574
41 	Santo Antônio da Platina 	46 251
42 	Campina Grande do Sul 	43 685
43 	Paiçandu 	41 773
44 	Dois Vizinhos 	41 038
45 	Jacarezinho 	39 322
46 	Guaratuba 	37 527
47 	Marialva 	35 804
48 	Matinhos 	35 219
49 	Jaguariaíva 	35 027
50 	Mandaguari 	34 515
51 	Rio Negro 	34 411
52 	Quedas do Iguaçu 	34 409
53 	Palmeira 	33 994
54 	Assis Chateaubriand 	33 340
55 	Guaíra 	33 310
56 	Imbituva 	32 940
57 	Pinhão 	32 559
58 	Rio Branco do Sul 	32 517
59 	Laranjeiras do Sul 	32 139
60 	Palotina 	32 121
61 	Ivaiporã 	31 935
62 	Ibaiti 	31 644
63 	Bandeirantes 	31 211
64 	Pitanga 	29 994
65 	Campo Magro 	29 740
66 	Itaperuçu 	29 070
67 	Goioerê 	28 808
68 	Arapoti 	28 300
69 	Nova Esperança 	27 984
70 	Pontal do Paraná 	27 915
71 	São Miguel do Iguaçu 	27 576
72 	Mandirituba 	27 315
73 	Reserva 	26 825
74 	Santa Helena 	26 767
75 	Astorga 	26 209
76 	Piraí do Sul 	25 617
77 	Cambará 	25 466
78 	Colorado 	24 145
79 	Quatro Barras 	23 911
80 	Carambeí 	23 825
81 	Santa Terezinha de Itaipu 	23 699
82 	Loanda 	23 242
83 	Mandaguaçu 	23 100
84 	Altônia 	22 176
85 	Ortigueira 	21 960
86 	Siqueira Campos 	21 249
87 	Jandaia do Sul 	21 230
88 	Cruzeiro do Oeste 	20 947
89 	Ubiratã 	20 909
90 	Tibagi 	20 607
91 	Coronel Vivida 	20 580
92 	Santo Antônio do Sudoeste 	20 261
93 	Andirá 	19 926
94 	Wenceslau Braz 	19 386
95 	Sengés 	19 385
96 	Ampére 	19 311
97 	Quitandinha 	19 221
98 	Chopinzinho 	19 167
99 	Capanema 	19 148
100 	Antonina 	18 949
101 	Contenda 	18 837
102 	Cruz Machado 	18 741
103 	Cafelândia 	18 456
104 	Matelândia 	18 107
105 	Cerro Azul 	17 833
106 	Terra Roxa 	17 522
107 	Faxinal 	17 316
108 	Terra Boa 	17 200
109 	Corbélia 	17 117
110 	Tijucas do Sul 	17 084
111 	Realeza 	16 950
112 	Terra Rica 	16 924
113 	Mangueirinha 	16 642
114 	Clevelândia 	16 450
115 	Morretes 	16 446
116 	Sertanópolis 	16 413
117 	Bituruna 	16 400
118 	Tapejara 	16 345
119 	Candói 	16 053
120 	Capitão Leônidas Marques 	15 834
121 	Bela Vista do Paraíso 	15 399
122 	Rio Azul 	15 336
123 	Ipiranga 	15 251
124 	São João do Triunfo 	15 241
125 	Curiúva 	15 196
126 	Tamarana 	15 040
127 	Assaí 	14 954
128 	Rebouças 	14 946
129 	Salto do Lontra 	14 872
130 	Alto Paraná 	14 859
131 	Cândido de Abreu 	14 809
132 	Santa Izabel do Oeste 	14 794
133 	Marmeleiro 	14 387
134 	Carlópolis 	14 356
135 	Campina da Lagoa 	14 043
136 	Paraíso do Norte 	14 023
137 	Peabiru 	14 007
138 	Araruna 	14 000
139 	Engenheiro Beltrão 	13 981
140 	Ivaí 	13 965
141 	Iporã 	13 782
142 	Jaguapitã 	13 742
143 	General Carneiro 	13 685
144 	Mallet 	13 663
145 	Manoel Ribas 	13 510
146 	Planalto 	13 431
147 	Cantagalo 	13 329
148 	Imbaú 	13 282
149 	Rio Bonito do Iguaçu 	13 255
150 	Nova Londrina 	13 200
151 	Bocaiuva do Sul 	13 129
152 	Turvo 	13 095
153 	Balsa Nova 	13 092
154 	Mamborê 	13 014
155 	Palmital 	12 960
156 	Ribeirão do Pinhal 	12 948
157 	Piên 	12 882
158 	Cidade Gaúcha 	12 797
159 	Porecatu 	12 748
160 	Jataizinho 	12 638
161 	Teixeira Soares 	12 567
162 	Querência do Norte 	12 232
163 	Guaraniaçu 	12 217
164 	Santa Fé 	12 186
165 	Itapejara d'Oeste 	12 094
166 	Ventania 	12 088
167 	Moreira Sales 	12 042
168 	Três Barras do Paraná 	12 038
169 	Joaquim Távora 	12 009
170 	Céu Azul 	11 819
171 	Santa Mariana 	11 622
172 	Paranacity 	11 580
173 	Nova Laranjeiras 	11 507
174 	Alvorada do Sul 	11 503
175 	Barbosa Ferraz 	11 426
176 	Itaipulândia 	11 385
177 	Pérola 	11 321
178 	Uraí 	11 273
179 	Jardim Alegre 	11 196
180 	Primeiro de Maio 	11 130
181 	São Jerônimo da Serra 	11 128
182 	Inácio Martins 	11 121
183 	São Pedro do Ivaí 	11 046
184 	Mauá da Serra 	10 800
185 	Centenário do Sul 	10 764
186 	Missal 	10 704
187 	Ribeirão Claro 	10 645
188 	Nova Prata do Iguaçu 	10 544
189 	Florestópolis 	10 453
190 	Mariluz 	10 336
191 	Barracão 	10 312
192 	Nova Aurora 	10 299
193 	São João 	10 181
194 	Catanduvas 	10 167
195 	Iretama 	10 098
196 	Santa Tereza do Oeste 	10 096
197 	São João do Ivaí 	10 056
198 	Alto Piquiri 	9 778
199 	Roncador 	9 645
200 	Rondon 	9 622
201 	Japurá 	9 500
202 	Agudos do Sul 	9 470
203 	Santa Maria do Oeste 	9 410
204 	São Jorge d'Oeste 	9 028
205 	Tunas do Paraná 	9 022
206 	Douradina 	8 869
207 	São Sebastião da Amoreira 	8 859
208 	Congonhinhas 	8 857
209 	Marilândia do Sul 	8 814
210 	Guamiranga 	8 811
211 	Califórnia 	8 606
212 	Tuneiras do Oeste 	8 533
213 	Santa Isabel do Ivaí 	8 523
214 	Vera Cruz do Oeste 	8 454
215 	Jesuítas 	8 330
216 	Nova Santa Rosa 	8 266
217 	Ivaté 	8 240
218 	Nova Fátima 	8 136
219 	Tupãssi 	8 109
220 	Reserva do Iguaçu 	8 069
221 	Campo do Tenente 	8 045
222 	Cambira 	7 917
223 	Tomazina 	7 807
224 	Icaraíma 	7 786
225 	Santa Cruz de Monte Castelo 	7 751
226 	Figueira 	7 696
227 	Guaraqueçaba 	7 594
228 	Boa Vista da Aparecida 	7 540
229 	Coronel Domingos Soares 	7 518
230 	Quatiguá 	7 477
231 	Antônio Olinto 	7 427
232 	Abatiá 	7 408
233 	Paulo Frontin 	7 387
234 	Juranda 	7 292
235 	Luiziana 	7 240
236 	Verê 	7 174
237 	Marilena 	7 084
238 	Bom Sucesso 	7 068
239 	Goioxim 	7 053
240 	Jussara 	7 041
241 	São Carlos do Ivaí 	6 920
242 	Sabáudia 	6 891
243 	Vitorino 	6 859
244 	Floresta 	6 851
245 	Renascença 	6 787
246 	Sapopema 	6 722
247 	Mariópolis 	6 632
248 	Guairaçá 	6 609
249 	Itambaracá 	6 549
250 	Formosa do Oeste 	6 460
251 	Borrazópolis 	6 439
252 	Ibema 	6 370
253 	Boa Ventura de São Roque 	6 365
254 	Amaporã 	6 332
255 	Pinhalão 	6 324
256 	Pérola d'Oeste 	6 288
257 	Perobal 	6 160
258 	São José da Boa Vista 	6 160
259 	Itambé 	6 109
260 	Ouro Verde do Oeste 	6 016
261 	Francisco Alves 	5 993
262 	Doutor Camargo 	5 983
263 	Enéas Marques 	5 933
264 	Paula Freitas 	5 908
265 	Adrianópolis 	5 857
266 	São João do Caiuá 	5 837
267 	Nova Olímpia 	5 826
268 	São Pedro do Iguaçu 	5 820
269 	Laranjal 	5 784
270 	São Tomé 	5 750
271 	Pato Bragado 	5 684
272 	Maria Helena 	5 634
273 	Xambrê 	5 630
274 	Fernandes Pinheiro 	5 602
275 	São Jorge do Patrocínio 	5 586
276 	Maripá 	5 582
277 	Mercedes 	5 577
278 	Doutor Ulysses 	5 552
279 	São Jorge do Ivaí 	5 543
280 	Saudade do Iguaçu 	5 539
281 	Guaraci 	5 530
282 	Grandes Rios 	5 497
283 	Tapira 	5 495
284 	Nova Tebas 	5 448
285 	Santo Inácio 	5 416
286 	Braganey 	5 382
287 	Presidente Castelo Branco 	5 351
288 	Jaboti 	5 303
289 	Diamante d'Oeste 	5 266
290 	Sertaneja 	5 216
291 	Tamboara 	5 158
292 	Honório Serpa 	5 119
293 	Janiópolis 	5 095
294 	Pranchita 	5 095
295 	Nova Cantu 	5 061
296 	Diamante do Norte 	5 030
297 	Nova Esperança do Sudoeste 	5 030
298 	Santana do Itararé 	4 954
299 	Lupionópolis 	4 945
300 	Japira 	4 930
301 	Floraí 	4 906
302 	Salto do Itararé 	4 898
303 	Porto Amazonas 	4 874
304 	Lobato 	4 819
305 	Fênix 	4 748
306 	Lunardelli 	4 744
307 	Rosário do Ivaí 	4 689
308 	Marumbi 	4 677
309 	Flor da Serra do Sul 	4 603
310 	Entre Rios do Oeste 	4 596
311 	Lindoeste 	4 592
312 	Foz do Jordão 	4 556
313 	Quinta do Sol 	4 508
314 	Serranópolis do Iguaçu 	4 477
315 	Ramilândia 	4 476
316 	Indianópolis 	4 465
317 	Quarto Centenário 	4 465
318 	Cruzeiro do Sul 	4 449
319 	Itaguajé 	4 446
320 	Iguaraçu 	4 440
321 	Marquinho 	4 340
322 	Nova Santa Bárbara 	4 277
323 	Planaltina do Paraná 	4 272
324 	Cruzeiro do Iguaçu 	4 240
325 	Rio Branco do Ivaí 	4 109
326 	Porto Vitória 	4 061
327 	Espigão Alto do Iguaçu 	4 048
328 	Boa Esperança 	4 047
329 	Kaloré 	4 047
330 	Quatro Pontes 	4 029
331 	Virmond 	4 022
332 	Santa Mônica 	4 017
333 	Cafezal do Sul 	4 009
334 	Munhoz de Melo 	4 009
335 	Nossa Senhora das Graças 	4 008
336 	Leópolis 	3 925
337 	Atalaia 	3 881
338 	Conselheiro Mairinck 	3 876
339 	Campina do Simão 	3 859
340 	Santa Lúcia 	3 793
341 	Guapirama 	3 784
342 	Rancho Alegre 	3 784
343 	Prado Ferreira 	3 780
344 	Campo Bonito 	3 763
345 	São José das Palmeiras 	3 627
346 	Bom Jesus do Sul 	3 506
347 	Salgado Filho 	3 483
348 	Bela Vista da Caroba 	3 457
349 	Nova América da Colina 	3 434
350 	Ourizona 	3 425
351 	Diamante do Sul 	3 424
352 	Santa Cecília do Pavão 	3 293
353 	Ivatuba 	3 279
354 	Jundiaí do Sul 	3 269
355 	Santa Amélia 	3 266
356 	Pitangueiras 	3 262
357 	Bom Sucesso do Sul 	3 254
358 	Paranapoema 	3 241
359 	Lidianópolis 	3 231
360 	Mato Rico 	3 206
361 	Porto Barreiro 	3 184
362 	Rio Bom 	3 182
363 	Corumbataí do Sul 	3 127
364 	Inajá 	3 116
365 	Farol 	3 041
366 	Arapuã 	3 009
367 	Cafeara 	2 954
368 	Ângulo 	2 930
369 	Sulina 	2 930
370 	Cruzmaltina 	2 921
371 	Godoy Moreira 	2 898
372 	Novo Itacolomi 	2 840
373 	Anahy 	2 788
374 	Barra do Jacaré 	2 781
375 	Itaúna do Sul 	2 781
376 	Pinhal de São Bento 	2 737
377 	Flórida 	2 699
378 	Alto Paraíso 	2 685
379 	Rancho Alegre d'Oeste 	2 628
380 	Santo Antônio do Caiuá 	2 626
381 	Uniflor 	2 614
382 	Brasilândia do Sul 	2 585
383 	Porto Rico 	2 556
384 	Manfrinópolis 	2 506
385 	Boa Esperança do Iguaçu 	2 470
386 	São Pedro do Paraná 	2 289
387 	Iguatu 	2 253
388 	Iracema do Oeste 	2 251
389 	Guaporema 	2 241
390 	Mirador 	2 196
391 	São Manoel do Paraná 	2 163
392 	Santo Antônio do Paraíso 	2 068
393 	Ariranha do Ivaí 	2 066
394 	Miraselva 	1 796
395 	Altamira do Paraná 	1 682
396 	Esperança Nova 	1 665
397 	Santa Inês 	1 594
398 	Nova Aliança do Ivaí 	1 551
399 	Jardim Olinda 	1 320
"""

In [136]:
from io import StringIO

In [137]:
populacao_pr = StringIO(populacao_pr)

In [138]:
populacao_pr = pd.read_csv(populacao_pr, sep='\t')

In [139]:
populacao_pr.columns

Index(['Posição ', 'Município ', 'População'], dtype='object')

In [140]:
from unidecode import unidecode

In [141]:
colunas = []

for coluna in populacao_pr.columns:
    colunas.append(unidecode(coluna.upper()))

populacao_pr.columns = colunas

In [145]:
populacao_pr.iloc[25,:] = [26,  'Pato Branco', '83 843']
populacao_pr.iloc[25,:]

POSICAO                26
MUNICIPIO     Pato Branco
POPULACAO          83 843
Name: 25, dtype: object

In [151]:
pop_pr = []

for pop in populacao_pr['POPULACAO']:
    pop_pr.append(int(pop.replace(' ','')))

populacao_pr['POPULACAO'] = pop_pr


In [152]:
populacao_pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   POSICAO     399 non-null    object
 1   MUNICIPIO   399 non-null    object
 2   POPULACAO   399 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 9.5+ KB
